In [16]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

In [17]:
train_series = pd.read_excel('train.xlsx')
test_series = pd.read_excel('test.xlsx')

In [18]:
X_train = train_series.set_index('ds').drop(columns=['y']).values  # Giriş (features) verisi
y_train = train_series.set_index('ds')['y'].values  # Çıkış (target) verisi

In [19]:
X_test = test_series.set_index('ds').values

In [20]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

In [21]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [22]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [23]:
test_dataset = TensorDataset(X_test_tensor)  # Test setinde y (hedef) olmadığı için sadece X var
test_loader = DataLoader(test_dataset, batch_size=32)

In [24]:
X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape

(torch.Size([15144, 24]), torch.Size([15144]), torch.Size([192, 24]))

In [ ]:
from transformers import PatchTSTConfig, PatchTSTForPrediction

config = PatchTSTConfig(
    patch_size=16,   
    hidden_size=512, 
    num_hidden_layers=4,
    num_attention_heads=8, 
    input_size=24,   
    prediction_length=24, 
    seq_length=168, 
)

In [34]:
model = PatchTSTForPrediction(config)

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss()

In [36]:
model.train()

PatchTSTForPrediction(
  (model): PatchTSTModel(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTStdScaler()
    )
    (patchifier): PatchTSTPatchify()
    (masking): Identity()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=1, out_features=512, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-3): 4 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): PatchTSTBatchNorm(
            (batchnorm): BatchNorm1d(512, eps=

In [ ]:
def train_model(model, train_loader, optimizer, loss_fn, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            
            outputs = model(X_batch).logits.squeeze()  # .logits -> PatchTST'nin çıktısı
            
            loss = loss_fn(outputs, y_batch)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}")

train_model(model, train_loader, optimizer, loss_fn, num_epochs=10)

IndexError: too many indices for tensor of dimension 2